In [ ]:
import pandas as pd
import numpy as np


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir("/content/drive/My Drive/CS5228 Project")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Train set basic statistics
df_train = pd.read_csv("train_coords.csv")
df_train["RESALE_PRICE"] = np.log(df_train["RESALE_PRICE"])

print("===== Data Overview =====")
print(df_train.info(), "\n")

for col in df_train.columns:
    print(f"--- Column: {col} ---")
    dtype = df_train[col].dtype
    print(f"Data Type: {dtype}")

    missing_count = df_train[col].isnull().sum()
    print(f"# Missing Values: {missing_count}")

    # Check if the column is numeric
    if np.issubdtype(dtype, np.number):
        # Remove missing values
        col_data = df_train[col].dropna()
        print("Type: Continuous (numeric)")
        print(f"Min: {col_data.min()}")
        print(f"Max: {col_data.max()}")
        print(f"Mean: {col_data.mean():.2f}")
        print(f"Std: {col_data.std():.2f}")
    else:
        # Treat as categorical
        unique_values = df_train[col].dropna().unique()
        print("Type: Categorical")
        print(f"Unique Values Count: {len(unique_values)}")
        print(f"Unique Values Set: {set(unique_values)}")

    print("\n")


===== Data Overview =====
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 162691 entries, 0 to 162690
Data columns (total 13 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   MONTH                162691 non-null  object 
 1   TOWN                 162691 non-null  object 
 2   FLAT_TYPE            162691 non-null  object 
 3   BLOCK                162691 non-null  object 
 4   STREET               162691 non-null  object 
 5   FLOOR_RANGE          162691 non-null  object 
 6   FLOOR_AREA_SQM       162691 non-null  float64
 7   FLAT_MODEL           162691 non-null  object 
 8   ECO_CATEGORY         162691 non-null  object 
 9   LEASE_COMMENCE_DATA  162691 non-null  int64  
 10  RESALE_PRICE         162691 non-null  float64
 11  LATITUDE             162691 non-null  float64
 12  LONGITUDE            162691 non-null  float64
dtypes: float64(4), int64(1), object(8)
memory usage: 16.1+ MB
None 

--- Column: MONTH ---
Data T

In [ ]:
# Test set basic statistics
df_test = pd.read_csv("test_coords.csv")

print("===== Data Overview =====")
print(df_test.info(), "\n")

for col in df_test.columns:
    print(f"--- Column: {col} ---")
    dtype = df_test[col].dtype
    print(f"Data Type: {dtype}")

    missing_count = df_test[col].isnull().sum()
    print(f"# Missing Values: {missing_count}")

    # Check if the column is numeric
    if np.issubdtype(dtype, np.number):
        # Remove missing values
        col_data = df_test[col].dropna()
        print("Type: Continuous (numeric)")
        print(f"Min: {col_data.min()}")
        print(f"Max: {col_data.max()}")
        print(f"Mean: {col_data.mean():.2f}")
        print(f"Std: {col_data.std():.2f}")
    else:
        # Treat as categorical
        unique_values = df_test[col].dropna().unique()
        print("Type: Categorical")
        print(f"Unique Values Count: {len(unique_values)}")
        print(f"Unique Values Set: {set(unique_values)}")

    print("\n")

===== Data Overview =====
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   MONTH                50000 non-null  object 
 1   TOWN                 50000 non-null  object 
 2   FLAT_TYPE            50000 non-null  object 
 3   BLOCK                50000 non-null  object 
 4   STREET               50000 non-null  object 
 5   FLOOR_RANGE          50000 non-null  object 
 6   FLOOR_AREA_SQM       50000 non-null  float64
 7   FLAT_MODEL           50000 non-null  object 
 8   ECO_CATEGORY         50000 non-null  object 
 9   LEASE_COMMENCE_DATA  50000 non-null  int64  
 10  LATITUDE             50000 non-null  float64
 11  LONGITUDE            50000 non-null  float64
dtypes: float64(3), int64(1), object(8)
memory usage: 4.6+ MB
None 

--- Column: MONTH ---
Data Type: object
# Missing Values: 0
Type: Categorical
Unique Values Coun

# Data Processing

## TOWN

In [ ]:
########## Target Encoding for TOWN (Out-of-Fold) ##########
from sklearn.model_selection import KFold

n_splits = 5
kf = KFold(n_splits=n_splits, shuffle=True, random_state=0)

# Create new column for target encoding
df_train["TOWN_TARGET_ENC"] = np.nan

# Cross-validation loop
for fold, (train_idx, val_idx) in enumerate(kf.split(df_train)):
    train_fold = df_train.iloc[train_idx]
    val_fold = df_train.iloc[val_idx]

    # Calculate the mean RESALE_PRICE for each TOWN in the current fold's training set
    fold_mean_map = train_fold.groupby("TOWN")["RESALE_PRICE"].mean().to_dict()

    # Map the means to the validation set
    df_train.loc[val_idx, "TOWN_TARGET_ENC"] = val_fold["TOWN"].map(fold_mean_map)

    # Fill unseen TOWN in validation set with global mean
    fold_global_mean = train_fold["RESALE_PRICE"].mean()
    df_train.loc[val_idx, "TOWN_TARGET_ENC"] = df_train.loc[val_idx, "TOWN_TARGET_ENC"].fillna(fold_global_mean)

# Create mapping for test set, using entire training set
town_mean_map = df_train.groupby("TOWN")["RESALE_PRICE"].mean().to_dict()
global_mean = df_train["RESALE_PRICE"].mean()

df_test["TOWN_TARGET_ENC"] = df_test["TOWN"].map(town_mean_map)
df_test["TOWN_TARGET_ENC"] = df_test["TOWN_TARGET_ENC"].fillna(global_mean)

print("Safe target encoding (no leakage) applied on 'TOWN'.")
print(df_train[["TOWN", "TOWN_TARGET_ENC"]].head())
print(df_test[["TOWN", "TOWN_TARGET_ENC"]].head())


In [ ]:
# ########## Target Encoding for TOWN ##########

# # Compute mean RESALE_PRICE for each TOWN in the training set
# town_mean_map = df_train.groupby("TOWN")["RESALE_PRICE"].mean().to_dict()

# # Apply the mapping to both train and test sets
# df_train["TOWN_TARGET_ENC"] = df_train["TOWN"].map(town_mean_map)
# df_test["TOWN_TARGET_ENC"]  = df_test["TOWN"].map(town_mean_map)

# # Apply global mean for unseen categories in test set
# global_mean = df_train["RESALE_PRICE"].mean()
# df_test["TOWN_TARGET_ENC"] = df_test["TOWN_TARGET_ENC"].fillna(global_mean)

# print("Target encoding applied on 'TOWN'.")
# print(df_train[["TOWN_TARGET_ENC"]].head())
# print(df_test[["TOWN_TARGET_ENC"]].head())

Target encoding applied on 'TOWN'.
   TOWN_TARGET_ENC
0        13.018201
1        13.409189
2        13.072957
3        13.166479
4        13.099824
   TOWN_TARGET_ENC
0        13.073720
1        13.275217
2        13.409189
3        13.099824
4        13.072957


In [ ]:
# ########## One-Hot Encoding for "TOWN" column ##########

# from sklearn.preprocessing import OneHotEncoder

# town_encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False) # ignore: unseen categories in test set -> all zeros

# town_one_hot = town_encoder.fit_transform(df_train[["TOWN"]])

# town_one_hot_df = pd.DataFrame(
#     town_one_hot,
#     columns=town_encoder.get_feature_names_out(["TOWN"]),
#     index=df_train.index
# )

# df_train = pd.concat([df_train, town_one_hot_df], axis=1) # do not drop "TOWN", as it may be needed for other encodings

# print(len(df_train.columns))

In [ ]:
# # Apply the same transformation to the test set
# df_test_town_one_hot = town_encoder.transform(df_test[["TOWN"]])

# df_test_town_one_hot_df = pd.DataFrame(
#     df_test_town_one_hot,
#     columns=town_encoder.get_feature_names_out(["TOWN"]),
#     index=df_test.index
# )


# df_test = pd.concat([df_test, df_test_town_one_hot_df], axis=1)

# print(len(df_test.columns))

## BLOCK

In [ ]:
########## Target Encoding for (STREET, BLOCK) ##########
from sklearn.model_selection import KFold

n_splits = 5
kf = KFold(n_splits=n_splits, shuffle=True, random_state=0)

df_train["STREET_BLOCK_TARGET_ENC"] = np.nan

# Out-of-Fold target encoding
for fold, (train_idx, val_idx) in enumerate(kf.split(df_train)):
    train_fold = df_train.iloc[train_idx]
    val_fold = df_train.iloc[val_idx]

    # Compute mean RESALE_PRICE for each (STREET, BLOCK) pair in training fold
    fold_mean_map = (
        train_fold.groupby(["STREET", "BLOCK"])["RESALE_PRICE"]
        .mean()
        .to_dict()
    )

    # Map the means to the validation fold
    val_enc = val_fold.set_index(["STREET", "BLOCK"]).index.map(fold_mean_map)
    df_train.loc[val_idx, "STREET_BLOCK_TARGET_ENC"] = val_enc

    # Fill unseen (STREET, BLOCK) pairs with TOWN-level mean
    fold_town_mean = train_fold.groupby("TOWN")["RESALE_PRICE"].mean().to_dict()
    df_train.loc[val_idx, "STREET_BLOCK_TARGET_ENC"] = df_train.loc[val_idx, "STREET_BLOCK_TARGET_ENC"].fillna(
        val_fold["TOWN"].map(fold_town_mean)
    )

    # Fill remaining missing values with global mean
    fold_global_mean = train_fold["RESALE_PRICE"].mean()
    df_train.loc[val_idx, "STREET_BLOCK_TARGET_ENC"] = df_train.loc[val_idx, "STREET_BLOCK_TARGET_ENC"].fillna(fold_global_mean)

# Create mapping for test set (using all training data)
street_block_mean_map = (
    df_train.groupby(["STREET", "BLOCK"])["RESALE_PRICE"]
    .mean()
    .to_dict()
)

# Map test set
df_test["STREET_BLOCK_TARGET_ENC"] = df_test.set_index(["STREET", "BLOCK"]).index.map(street_block_mean_map)

# Fill unseen (STREET, BLOCK) pairs in test set with TOWN-level mean
town_mean_map = df_train.groupby("TOWN")["RESALE_PRICE"].mean().to_dict()
df_test["STREET_BLOCK_TARGET_ENC"] = df_test["STREET_BLOCK_TARGET_ENC"].fillna(df_test["TOWN"].map(town_mean_map))

# Fill remaining missing values with global mean
global_mean = df_train["RESALE_PRICE"].mean()
df_test["STREET_BLOCK_TARGET_ENC"] = df_test["STREET_BLOCK_TARGET_ENC"].fillna(global_mean)

print("Safe target encoding (no leakage) applied on (STREET, BLOCK).")
print(df_train[["STREET", "BLOCK", "TOWN", "STREET_BLOCK_TARGET_ENC"]].head())
print(df_test[["STREET", "BLOCK", "TOWN", "STREET_BLOCK_TARGET_ENC"]].head())


## FLAT_TYPE

In [ ]:
########## Ordinal + Target Encoding for "FLAT_TYPE" column ##########
flat_type_order = {
    "1 ROOM": 1,
    "2 ROOM": 2,
    "3 ROOM": 3,
    "4 ROOM": 4,
    "5 ROOM": 5,
    "EXECUTIVE": 6,
    "MULTI GENERATION": 7
}

df_train["FLAT_TYPE"] = (
    df_train["FLAT_TYPE"].astype(str).str.replace("-", " ", regex=False)
)
df_test["FLAT_TYPE"] = (
    df_test["FLAT_TYPE"].astype(str).str.replace("-", " ", regex=False)
)

df_train["FLAT_TYPE_ENC"] = df_train["FLAT_TYPE"].str.upper().map(flat_type_order)
df_test["FLAT_TYPE_ENC"]  = df_test["FLAT_TYPE"].str.upper().map(flat_type_order)

print(df_train[["FLAT_TYPE", "FLAT_TYPE_ENC"]].head())

  FLAT_TYPE  FLAT_TYPE_ENC
0    4 room              4
1    4 room              4
2    4 room              4
3    4 room              4
4    5 room              5


In [ ]:
n_splits = 5
kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

df_train["FLAT_TYPE_TARGET_ENC"] = np.nan

# Out-of-Fold target encoding
for fold, (train_idx, val_idx) in enumerate(kf.split(df_train)):
    train_fold = df_train.iloc[train_idx]
    val_fold = df_train.iloc[val_idx]

    # Calculate fold mean mapping
    fold_mean_map = train_fold.groupby("FLAT_TYPE")["RESALE_PRICE"].mean().to_dict()

    # Map the means to the validation set
    val_enc = val_fold["FLAT_TYPE"].map(fold_mean_map)

    # Fill unseen types with the current fold's global mean
    fold_global_mean = train_fold["RESALE_PRICE"].mean()
    val_enc = val_enc.fillna(fold_global_mean)

    # Write the results back to the main dataframe
    df_train.loc[val_idx, "FLAT_TYPE_TARGET_ENC"] = val_enc

# Create mapping for test set, using entire training set
flat_type_mean_map = df_train.groupby("FLAT_TYPE")["RESALE_PRICE"].mean().to_dict()
global_mean = df_train["RESALE_PRICE"].mean()

# Map the means to the test set
df_test["FLAT_TYPE_TARGET_ENC"] = df_test["FLAT_TYPE"].map(flat_type_mean_map)
df_test["FLAT_TYPE_TARGET_ENC"] = df_test["FLAT_TYPE_TARGET_ENC"].fillna(global_mean)

print("Safe target encoding (no leakage) applied on 'FLAT_TYPE'.")
print(df_train[["FLAT_TYPE", "FLAT_TYPE_TARGET_ENC"]].head())
print(df_test[["FLAT_TYPE", "FLAT_TYPE_TARGET_ENC"]].head())

## MONTH

In [ ]:
df_train["YEAR_INT"] = df_train["MONTH"].str[:4].astype(int)
df_train["MONTH_INT"] = df_train["MONTH"].str[-2:].astype(int)

print(df_train[["MONTH", "YEAR_INT", "MONTH_INT"]].head())

     MONTH  YEAR_INT  MONTH_INT
0  2020-10      2020         10
1  2021-07      2021          7
2  2021-05      2021          5
3  2021-08      2021          8
4  2023-05      2023          5


In [ ]:
df_test["YEAR_INT"] = df_test["MONTH"].str[:4].astype(int)
df_test["MONTH_INT"] = df_test["MONTH"].str[-2:].astype(int)

print(df_test[["MONTH", "YEAR_INT", "MONTH_INT"]].head())

     MONTH  YEAR_INT  MONTH_INT
0  2025-06      2025          6
1  2020-01      2020          1
2  2025-06      2025          6
3  2022-10      2022         10
4  2024-02      2024          2


## FLAT_MODEL

In [ ]:
from sklearn.preprocessing import OneHotEncoder

########## One-Hot Encoding for FLAT_MODEL ##########

flat_model_encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False) # ignore: unseen categories in test set -> all zeros
flat_model_one_hot_train = flat_model_encoder.fit_transform(df_train[["FLAT_MODEL"]])
flat_model_one_hot_test = flat_model_encoder.transform(df_test[["FLAT_MODEL"]])

flat_model_one_hot_train_df = pd.DataFrame(
    flat_model_one_hot_train,
    columns=flat_model_encoder.get_feature_names_out(["FLAT_MODEL"]),
    index=df_train.index
)

flat_model_one_hot_test_df = pd.DataFrame(
    flat_model_one_hot_test,
    columns=flat_model_encoder.get_feature_names_out(["FLAT_MODEL"]),
    index=df_test.index
)

df_train = pd.concat([df_train, flat_model_one_hot_train_df], axis=1)
df_test  = pd.concat([df_test,  flat_model_one_hot_test_df], axis=1)

print("One-hot encoding applied on 'FLAT_MODEL'.")
print(df_train.filter(like="FLAT_MODEL_").head())
print(df_test.filter(like="FLAT_MODEL_").head())


One-hot encoding applied on 'FLAT_MODEL'.
   FLAT_MODEL_2 room  FLAT_MODEL_3gen  FLAT_MODEL_adjoined flat  \
0                0.0              0.0                       0.0   
1                0.0              0.0                       0.0   
2                0.0              0.0                       0.0   
3                0.0              0.0                       0.0   
4                0.0              0.0                       0.0   

   FLAT_MODEL_apartment  FLAT_MODEL_dbss  FLAT_MODEL_improved  \
0                   0.0              0.0                  0.0   
1                   0.0              0.0                  0.0   
2                   0.0              0.0                  0.0   
3                   0.0              0.0                  0.0   
4                   0.0              0.0                  1.0   

   FLAT_MODEL_improved maisonette  FLAT_MODEL_maisonette  FLAT_MODEL_model a  \
0                             0.0                    0.0                 0.0   
1   

In [ ]:
###### Target Encoding ######
def te_cv_train(df, col="FLAT_MODEL", target_col="RESALE_PRICE",
                n_splits=5, out_col=None, random_state=0):
    """
    K-Fold CV Target Encoding.
    """
    df = df.copy()
    if out_col is None:
        out_col = f"{col}_TARGET_ENC"
    df[out_col] = pd.NA

    kf = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    for tr_idx, va_idx in kf.split(df):
        tr = df.iloc[tr_idx]
        va = df.iloc[va_idx]

        mean_map = tr.groupby(col)[target_col].mean().to_dict()
        s = df.loc[va.index, col].map(mean_map)
        fold_global = tr[target_col].mean()
        s = s.fillna(fold_global)

        df.loc[va.index, out_col] = s

    df[out_col] = df[out_col].astype(float)
    df[out_col] = df[out_col].fillna(df[target_col].mean())

    return df

def build_te_mapping(df_train, col="FLAT_MODEL", target_col="RESALE_PRICE"):
    mean_map = df_train.groupby(col)[target_col].mean().to_dict()
    global_mean = df_train[target_col].mean()
    return mean_map, global_mean

def te_apply_test(df_test, mean_map, global_mean, col="FLAT_MODEL", out_col="FLAT_MODEL_TARGET_ENC"):
    df = df_test.copy()
    s = df[col].map(mean_map).fillna(global_mean)
    df[out_col] = s.astype(float)
    return df

mean_map, global_mean = build_te_mapping(df_train, col="FLAT_MODEL", target_col="RESALE_PRICE")

df_train = te_cv_train(df_train, col="FLAT_MODEL", target_col="RESALE_PRICE", n_splits=5, out_col="FLAT_MODEL_TARGET_ENC")
df_test = te_apply_test(df_test, mean_map, global_mean, col="FLAT_MODEL", out_col="FLAT_MODEL_TARGET_ENC")

print(df_train[["FLAT_MODEL", "FLAT_MODEL_TARGET_ENC"]].head())
print(df_test[["FLAT_MODEL", "FLAT_MODEL_TARGET_ENC"]].head())

## FLOOR_RANGE

In [ ]:
import re
from bisect import bisect_left

_rng = re.compile(r'(\d+)\s*to\s*(\d+)', flags=re.IGNORECASE)

def parse_floor_range(x):
    if pd.isna(x):
        return None
    s = str(x).strip()
    m = _rng.search(s)
    if not m:
        return None
    lo, hi = int(m.group(1)), int(m.group(2))
    if lo > hi:
        lo, hi = hi, lo
    return lo, hi

def build_floor_order_mapping(df_train, col="FLOOR_RANGE"):
    pairs = (
        df_train[col]
        .dropna()
        .map(parse_floor_range)
        .dropna()
        .unique()
    )
    pairs = sorted(pairs, key=lambda t: (t[0], t[1]))
    mapping = {f"{lo:02d} to {hi:02d}": i for i, (lo, hi) in enumerate(pairs)}
    for i, (lo, hi) in enumerate(pairs):
        mapping[f"{lo} to {hi}"] = i

    lows = [lo for (lo, _) in pairs]
    median_code = int(np.median(list(mapping.values()))) if mapping else 0
    return mapping, pairs, lows, median_code

def encode_floor_range_series(sr, mapping, pairs, lows, median_code):
    enc = sr.map(mapping)
    mask_na = enc.isna()
    if mask_na.any():
        parsed = sr[mask_na].map(parse_floor_range)

        def fallback_code(p):
            if p is None:
                return median_code
            lo, hi = p
            if not lows:
                return median_code
            pos = bisect_left(lows, lo)
            pos = max(0, min(pos, len(pairs)-1))
            return pos

        enc.loc[mask_na] = parsed.map(fallback_code)

    return enc.astype(int)


mapping, pairs, lows, median_code = build_floor_order_mapping(df_train, col="FLOOR_RANGE")


df_train["FLOOR_RANGE_ORD"] = encode_floor_range_series(
    df_train["FLOOR_RANGE"], mapping, pairs, lows, median_code
)


df_test["FLOOR_RANGE_ORD"] = encode_floor_range_series(
    df_test["FLOOR_RANGE"], mapping, pairs, lows, median_code
)

print(df_train[["FLOOR_RANGE", "FLOOR_RANGE_ORD"]].head())
print(df_test[["FLOOR_RANGE", "FLOOR_RANGE_ORD"]].head())


  FLOOR_RANGE  FLOOR_RANGE_ORD
0    07 to 09                2
1    07 to 09                2
2    19 to 21                6
3    16 to 18                5
4    10 to 12                3
  FLOOR_RANGE  FLOOR_RANGE_ORD
0    04 to 06                1
1    01 to 03                0
2    10 to 12                3
3    10 to 12                3
4    10 to 12                3


## LEASE_COMMENCE_DATE

In [ ]:
########## Compute flat age (year difference) ##########
df_train["YEAR_INT"] = pd.to_numeric(df_train["YEAR_INT"], errors="coerce")
df_train["LEASE_COMMENCE_DATA"] = pd.to_numeric(df_train["LEASE_COMMENCE_DATA"], errors="coerce")
df_test["YEAR_INT"] = pd.to_numeric(df_test["YEAR_INT"], errors="coerce")
df_test["LEASE_COMMENCE_DATA"] = pd.to_numeric(df_test["LEASE_COMMENCE_DATA"], errors="coerce")

df_train["LEASE_AGE"] = df_train["YEAR_INT"] - df_train["LEASE_COMMENCE_DATA"]
df_test["LEASE_AGE"]  = df_test["YEAR_INT"]  - df_test["LEASE_COMMENCE_DATA"]

print("New column 'LEASE_AGE' created.")
print(df_train[["YEAR_INT", "LEASE_COMMENCE_DATA", "LEASE_AGE"]].head())
print(df_test[["YEAR_INT", "LEASE_COMMENCE_DATA", "LEASE_AGE"]].head())

New column 'LEASE_AGE' created.
   YEAR_INT  LEASE_COMMENCE_DATA  LEASE_AGE
0      2020                 2000         20
1      2021                 1992         29
2      2021                 1998         23
3      2021                 2017          4
4      2023                 2018          5
   YEAR_INT  LEASE_COMMENCE_DATA  LEASE_AGE
0      2025                 2018          7
1      2020                 2003         17
2      2025                 1973         52
3      2022                 1987         35
4      2024                 1988         36


## Hawkers

In [ ]:
########## Hawker Distance Features (Shared Distance Matrix + TopK Aggregation) ##########
from tqdm import tqdm

hawkers = pd.read_csv("auxiliary-data/sg-gov-hawkers.csv")
hawkers["LATITUDE"] = pd.to_numeric(hawkers["LATITUDE"], errors="coerce")
hawkers["LONGITUDE"] = pd.to_numeric(hawkers["LONGITUDE"], errors="coerce")
hawkers = hawkers.dropna(subset=["LATITUDE", "LONGITUDE"]).reset_index(drop=True)

stall_col = "NUMBER_OF_STALLS"
lat_col, lon_col = "LATITUDE", "LONGITUDE"

def compute_distance_matrix(df, ref_df, lat_col="LATITUDE", lon_col="LONGITUDE"):
    dist_matrix = np.zeros((len(df), len(ref_df)))
    for i, row in tqdm(enumerate(df.itertuples()), total=len(df), desc=f"Computing {len(df)}×{len(ref_df)} distances"):
        lat, lon = getattr(row, lat_col), getattr(row, lon_col)
        dists = np.sqrt((ref_df["LATITUDE"] - lat)**2 + (ref_df["LONGITUDE"] - lon)**2)
        dist_matrix[i, :] = dists.values
    return dist_matrix


print("Computing distance matrices for train and test...")
dist_train = compute_distance_matrix(df_train, hawkers)
dist_test  = compute_distance_matrix(df_test,  hawkers)
print("Distance matrices computed once.\n")


def add_basic_hawker_features(df, dist_matrix, hawkers):
    thres_500 = 0.0045
    thres_1000 = 0.009
    stalls = hawkers[stall_col].values.reshape(1, -1)

    df = df.copy()
    df["DIST_NEAREST_HAWKER"] = dist_matrix.min(axis=1)
    df["DIST_SECOND_HAWKER"]  = np.partition(dist_matrix, 1, axis=1)[:, 1]
    df["NUM_HAWKER_500M"]     = (dist_matrix < thres_500).sum(axis=1)
    df["NUM_HAWKER_1000M"]    = (dist_matrix < thres_1000).sum(axis=1)
    df["HAWKER_STALLS_500M"]  = ((dist_matrix < thres_500) * stalls).sum(axis=1)
    df["HAWKER_STALLS_1000M"] = ((dist_matrix < thres_1000) * stalls).sum(axis=1)

    return df


df_train = add_basic_hawker_features(df_train, dist_train, hawkers)
df_test  = add_basic_hawker_features(df_test,  dist_test,  hawkers)
print("Basic hawker features added.\n")

print("Computing hawker–price correlations (train only)...")
price = df_train["RESALE_PRICE"].values
inv_dist_train = 1 / (dist_train + 1e-8)
corrs = [np.corrcoef(inv_dist_train[:, j], price)[0, 1] for j in range(len(hawkers))]
hawkers["CORR_WITH_PRICE"] = corrs

hawkers_sorted = hawkers.sort_values("CORR_WITH_PRICE", ascending=False).reset_index(drop=True)
print("\nTop 10 hawkers most positively correlated with price:")
print(hawkers_sorted.head(10)[["NAME", "CORR_WITH_PRICE"]])


# Add the nearest distance feature of top3/top5/top10
def add_nearest_to_topk_hawker(df, dist_matrix, hawkers_sorted):
    df = df.copy()
    top3_idx = hawkers_sorted.index[:3]
    top5_idx = hawkers_sorted.index[:5]
    top10_idx = hawkers_sorted.index[:10]

    df["DIST_HAWKER_NEAREST_TOP3"]  = dist_matrix[:, top3_idx].min(axis=1)
    df["DIST_HAWKER_NEAREST_TOP5"]  = dist_matrix[:, top5_idx].min(axis=1)
    df["DIST_HAWKER_NEAREST_TOP10"] = dist_matrix[:, top10_idx].min(axis=1)
    return df


df_train = add_nearest_to_topk_hawker(df_train, dist_train, hawkers_sorted)
df_test  = add_nearest_to_topk_hawker(df_test,  dist_test,  hawkers_sorted)

print("Added nearest-distance-to-top3/5/10 hawker features.\n")

print(df_train[[
    "DIST_NEAREST_HAWKER", "DIST_SECOND_HAWKER",
    "NUM_HAWKER_500M", "NUM_HAWKER_1000M",
    "HAWKER_STALLS_500M", "HAWKER_STALLS_1000M",
    "DIST_HAWKER_NEAREST_TOP3", "DIST_HAWKER_NEAREST_TOP5", "DIST_HAWKER_NEAREST_TOP10"
]].head())
print(df_test[[
    "DIST_NEAREST_HAWKER", "DIST_SECOND_HAWKER",
    "NUM_HAWKER_500M", "NUM_HAWKER_1000M",
    "HAWKER_STALLS_500M", "HAWKER_STALLS_1000M",
    "DIST_HAWKER_NEAREST_TOP3", "DIST_HAWKER_NEAREST_TOP5", "DIST_HAWKER_NEAREST_TOP10"
]].head())


## MRT Stations

In [ ]:
########## MRT Accessibility Feature Engineering ##########

# Only consider open MRT stations
mrt = pd.read_csv("auxiliary-data/sg-mrt-stations.csv")
mrt_open = mrt[mrt["STATUS"].str.lower() == "open"].copy()

# Convert latitude and longitude to numeric, avoiding string type errors
mrt_open["LATITUDE"] = pd.to_numeric(mrt_open["LATITUDE"], errors="coerce")
mrt_open["LONGITUDE"] = pd.to_numeric(mrt_open["LONGITUDE"], errors="coerce")

# Latitude and longitude column names
lat_col, lon_col = "LATITUDE", "LONGITUDE"

# Initialize empty lists
nearest, second, count_500, count_1000 = [], [], [], []

# Iterate over each property record
for _, row in df_train.iterrows():
    lat, lon = row[lat_col], row[lon_col]

    # Calculate Euclidean distances to all MRT (open) stations (unit: degrees)
    dists = np.sqrt((mrt_open["LATITUDE"] - lat)**2 + (mrt_open["LONGITUDE"] - lon)**2)

    # Sort to get nearest and second nearest MRT
    sorted_dists = np.sort(dists.values)
    nearest.append(sorted_dists[0] if len(sorted_dists) > 0 else np.nan)
    second.append(sorted_dists[1] if len(sorted_dists) > 1 else np.nan)

    # Define thresholds (approx: 1° ≈ 111 km → 500m ≈ 0.0045°, 1000m ≈ 0.009°)
    thres_500 = 0.0045
    thres_1000 = 0.009

    # Calculate number of MRT stations within 500m and 1000m
    in_500 = dists < thres_500
    in_1000 = dists < thres_1000

    count_500.append(in_500.sum())
    count_1000.append(in_1000.sum())

# Add new feature columns
df_train["DIST_NEAREST_MRT_OPEN"] = nearest
df_train["DIST_SECOND_MRT_OPEN"] = second
df_train["NUM_MRT_500M_OPEN"] = count_500
df_train["NUM_MRT_1000M_OPEN"] = count_1000

print("New MRT (open) features added:")
print(df_train[[
    "DIST_NEAREST_MRT_OPEN",
    "DIST_SECOND_MRT_OPEN",
    "NUM_MRT_500M_OPEN",
    "NUM_MRT_1000M_OPEN"
]].head())


New MRT (open) features added:
   DIST_NEAREST_MRT_OPEN  DIST_SECOND_MRT_OPEN  NUM_MRT_500M_OPEN  \
0               0.002891              0.015468                  1   
1               0.003163              0.003163                  2   
2               0.009617              0.009617                  0   
3               0.007821              0.013880                  0   
4               0.012341              0.014990                  0   

   NUM_MRT_1000M_OPEN  
0                   1  
1                   3  
2                   0  
3                   1  
4                   0  


In [ ]:
########## MRT Accessibility Feature Engineering ##########

# Only consider open MRT stations
mrt = pd.read_csv("auxiliary-data/sg-mrt-stations.csv")
mrt_open = mrt[mrt["STATUS"].str.lower() == "open"].copy()

# Convert latitude and longitude to numeric, avoiding string type errors
mrt_open["LATITUDE"] = pd.to_numeric(mrt_open["LATITUDE"], errors="coerce")
mrt_open["LONGITUDE"] = pd.to_numeric(mrt_open["LONGITUDE"], errors="coerce")

# Latitude and longitude column names
lat_col, lon_col = "LATITUDE", "LONGITUDE"

# Initialize empty lists
nearest, second, count_500, count_1000 = [], [], [], []

# Iterate over each property record
for _, row in df_test.iterrows():
    lat, lon = row[lat_col], row[lon_col]

    # Calculate Euclidean distances to all MRT (open) stations (unit: degrees)
    dists = np.sqrt((mrt_open["LATITUDE"] - lat)**2 + (mrt_open["LONGITUDE"] - lon)**2)

    # Sort to get nearest and second nearest MRT
    sorted_dists = np.sort(dists.values)
    nearest.append(sorted_dists[0] if len(sorted_dists) > 0 else np.nan)
    second.append(sorted_dists[1] if len(sorted_dists) > 1 else np.nan)

    # Define thresholds (approx: 1° ≈ 111 km → 500m ≈ 0.0045°, 1000m ≈ 0.009°)
    thres_500 = 0.0045
    thres_1000 = 0.009

    # Calculate number of MRT stations within 500m and 1000m
    in_500 = dists < thres_500
    in_1000 = dists < thres_1000

    count_500.append(in_500.sum())
    count_1000.append(in_1000.sum())

# Add new feature columns
df_test["DIST_NEAREST_MRT_OPEN"] = nearest
df_test["DIST_SECOND_MRT_OPEN"] = second
df_test["NUM_MRT_500M_OPEN"] = count_500
df_test["NUM_MRT_1000M_OPEN"] = count_1000

print("New MRT (open) features added:")
print(df_test[[
    "DIST_NEAREST_MRT_OPEN",
    "DIST_SECOND_MRT_OPEN",
    "NUM_MRT_500M_OPEN",
    "NUM_MRT_1000M_OPEN"
]].head())


New MRT (open) features added:
   DIST_NEAREST_MRT_OPEN  DIST_SECOND_MRT_OPEN  NUM_MRT_500M_OPEN  \
0               0.002645              0.009580                  1   
1               0.004750              0.010154                  0   
2               0.005848              0.007062                  0   
3               0.011572              0.013169                  0   
4               0.012827              0.013200                  0   

   NUM_MRT_1000M_OPEN  
0                   1  
1                   1  
2                   3  
3                   0  
4                   0  


## Schools

In [ ]:
########## Primary School Accessibility Feature Engineering (Nearest-to-Top3/5/10) ##########

from tqdm import tqdm

schools = pd.read_csv("auxiliary-data/sg-primary-schools.csv")
schools["LATITUDE"] = pd.to_numeric(schools["LATITUDE"], errors="coerce")
schools["LONGITUDE"] = pd.to_numeric(schools["LONGITUDE"], errors="coerce")
schools = schools.dropna(subset=["LATITUDE", "LONGITUDE"]).reset_index(drop=True)

lat_col, lon_col = "LATITUDE", "LONGITUDE"

def compute_distance_matrix(df, ref_df):
    dist_matrix = np.zeros((len(df), len(ref_df)))
    for i, row in tqdm(enumerate(df.itertuples()), total=len(df), desc=f"Computing {len(df)}×{len(ref_df)} school distances"):
        lat, lon = getattr(row, lat_col), getattr(row, lon_col)
        dists = np.sqrt((ref_df["LATITUDE"] - lat)**2 + (ref_df["LONGITUDE"] - lon)**2)
        dist_matrix[i, :] = dists.values
    return dist_matrix

print("Computing distance matrices for train and test...")
dist_train_school = compute_distance_matrix(df_train, schools)
dist_test_school  = compute_distance_matrix(df_test,  schools)
print("Distance matrices computed.\n")

def add_basic_school_features(df, dist_matrix):
    thres_500 = 0.0045
    thres_1000 = 0.009
    df = df.copy()
    df["DIST_NEAREST_P_SCHOOL"]  = dist_matrix.min(axis=1)
    df["DIST_SECOND_P_SCHOOL"]   = np.partition(dist_matrix, 1, axis=1)[:, 1]
    df["NUM_P_SCHOOL_500M"]      = (dist_matrix < thres_500).sum(axis=1)
    df["NUM_P_SCHOOL_1000M"]     = (dist_matrix < thres_1000).sum(axis=1)
    return df

df_train = add_basic_school_features(df_train, dist_train_school)
df_test  = add_basic_school_features(df_test,  dist_test_school)
print("Basic school accessibility features added.\n")

print("Computing school–price correlations...")
price = df_train["RESALE_PRICE"].values
inv_dist_train = 1 / (dist_train_school + 1e-8)

corrs = [np.corrcoef(inv_dist_train[:, j], price)[0, 1] for j in range(len(schools))]
schools["CORR_WITH_PRICE"] = corrs

# Get Top 10
top_schools = schools.sort_values("CORR_WITH_PRICE", ascending=False).head(10).reset_index(drop=True)
print("\nTop 10 schools most positively correlated with resale price:")
print(top_schools[["NAME", "CORR_WITH_PRICE"]])


# Calculate the nearest distance of each house to these top schools (top3 / top5 / top10)
def add_nearest_to_topk_features(df, dist_matrix, schools, top_schools):
    df = df.copy()
    lat_all, lon_all = df["LATITUDE"].values, df["LONGITUDE"].values

    idx_top3 = top_schools.index[:3]
    idx_top5 = top_schools.index[:5]
    idx_top10 = top_schools.index[:10]

    dist_top3 = dist_matrix[:, idx_top3]
    dist_top5 = dist_matrix[:, idx_top5]
    dist_top10 = dist_matrix[:, idx_top10]

    # Take the minimum value of each row (i.e. the nearest distance from the topK school)
    df["DIST_P_SCHOOL_NEAREST_TOP3"] = dist_top3.min(axis=1)
    df["DIST_P_SCHOOL_NEAREST_TOP5"] = dist_top5.min(axis=1)
    df["DIST_P_SCHOOL_NEAREST_TOP10"] = dist_top10.min(axis=1)

    return df

df_train = add_nearest_to_topk_features(df_train, dist_train_school, schools, top_schools)
df_test  = add_nearest_to_topk_features(df_test,  dist_test_school,  schools, top_schools)

print("\nAdded nearest-distance-to-top3/5/10 school features to train and test sets.\n")

print(df_train[[
    "DIST_NEAREST_P_SCHOOL", "DIST_SECOND_P_SCHOOL",
    "NUM_P_SCHOOL_500M", "NUM_P_SCHOOL_1000M",
    "DIST_P_SCHOOL_NEAREST_TOP3", "DIST_P_SCHOOL_NEAREST_TOP5", "DIST_P_SCHOOL_NEAREST_TOP10"
]].head())
print(df_test[[
    "DIST_NEAREST_P_SCHOOL", "DIST_SECOND_P_SCHOOL",
    "NUM_P_SCHOOL_500M", "NUM_P_SCHOOL_1000M",
    "DIST_P_SCHOOL_NEAREST_TOP3", "DIST_P_SCHOOL_NEAREST_TOP5", "DIST_P_SCHOOL_NEAREST_TOP10"
]].head())


In [ ]:
########## Secondary School Accessibility Feature Engineering (Nearest-to-Top3/5/10) ##########

from tqdm import tqdm

schools = pd.read_csv("auxiliary-data/sg-secondary-schools.csv")
schools["LATITUDE"] = pd.to_numeric(schools["LATITUDE"], errors="coerce")
schools["LONGITUDE"] = pd.to_numeric(schools["LONGITUDE"], errors="coerce")
schools = schools.dropna(subset=["LATITUDE", "LONGITUDE"]).reset_index(drop=True)

lat_col, lon_col = "LATITUDE", "LONGITUDE"

def compute_distance_matrix(df, ref_df):
    dist_matrix = np.zeros((len(df), len(ref_df)))
    for i, row in tqdm(enumerate(df.itertuples()), total=len(df), desc=f"Computing {len(df)}×{len(ref_df)} school distances"):
        lat, lon = getattr(row, lat_col), getattr(row, lon_col)
        dists = np.sqrt((ref_df["LATITUDE"] - lat)**2 + (ref_df["LONGITUDE"] - lon)**2)
        dist_matrix[i, :] = dists.values
    return dist_matrix

print("Computing distance matrices for train and test...")
dist_train_school = compute_distance_matrix(df_train, schools)
dist_test_school  = compute_distance_matrix(df_test,  schools)
print("Distance matrices computed.\n")

def add_basic_school_features(df, dist_matrix):
    thres_500 = 0.0045
    thres_1000 = 0.009
    df = df.copy()
    df["DIST_NEAREST_S_SCHOOL"]  = dist_matrix.min(axis=1)
    df["DIST_SECOND_S_SCHOOL"]   = np.partition(dist_matrix, 1, axis=1)[:, 1]
    df["NUM_S_SCHOOL_500M"]      = (dist_matrix < thres_500).sum(axis=1)
    df["NUM_S_SCHOOL_1000M"]     = (dist_matrix < thres_1000).sum(axis=1)
    return df

df_train = add_basic_school_features(df_train, dist_train_school)
df_test  = add_basic_school_features(df_test,  dist_test_school)
print("Basic school accessibility features added.\n")

print("Computing school–price correlations...")
price = df_train["RESALE_PRICE"].values
inv_dist_train = 1 / (dist_train_school + 1e-8)

corrs = [np.corrcoef(inv_dist_train[:, j], price)[0, 1] for j in range(len(schools))]
schools["CORR_WITH_PRICE"] = corrs

# Get Top 10
top_schools = schools.sort_values("CORR_WITH_PRICE", ascending=False).head(10).reset_index(drop=True)
print("\nTop 10 schools most positively correlated with resale price:")
print(top_schools[["NAME", "CORR_WITH_PRICE"]])


# Calculate the nearest distance of each house to these top schools (top3 / top5 / top10)
def add_nearest_to_topk_features(df, dist_matrix, schools, top_schools):
    df = df.copy()
    lat_all, lon_all = df["LATITUDE"].values, df["LONGITUDE"].values

    idx_top3 = top_schools.index[:3]
    idx_top5 = top_schools.index[:5]
    idx_top10 = top_schools.index[:10]

    dist_top3 = dist_matrix[:, idx_top3]
    dist_top5 = dist_matrix[:, idx_top5]
    dist_top10 = dist_matrix[:, idx_top10]

    # Take the minimum value of each row (i.e. the nearest distance from the topK school)
    df["DIST_S_SCHOOL_NEAREST_TOP3"] = dist_top3.min(axis=1)
    df["DIST_S_SCHOOL_NEAREST_TOP5"] = dist_top5.min(axis=1)
    df["DIST_S_SCHOOL_NEAREST_TOP10"] = dist_top10.min(axis=1)

    return df

df_train = add_nearest_to_topk_features(df_train, dist_train_school, schools, top_schools)
df_test  = add_nearest_to_topk_features(df_test,  dist_test_school,  schools, top_schools)

print("\nAdded nearest-distance-to-top3/5/10 school features to train and test sets.\n")

print(df_train[[
    "DIST_NEAREST_S_SCHOOL", "DIST_SECOND_S_SCHOOL",
    "NUM_S_SCHOOL_500M", "NUM_S_SCHOOL_1000M",
    "DIST_S_SCHOOL_NEAREST_TOP3", "DIST_S_SCHOOL_NEAREST_TOP5", "DIST_S_SCHOOL_NEAREST_TOP10"
]].head())
print(df_test[[
    "DIST_NEAREST_S_SCHOOL", "DIST_SECOND_S_SCHOOL",
    "NUM_S_SCHOOL_500M", "NUM_S_SCHOOL_1000M",
    "DIST_S_SCHOOL_NEAREST_TOP3", "DIST_S_SCHOOL_NEAREST_TOP5", "DIST_S_SCHOOL_NEAREST_TOP10"
]].head())


## Malls

In [ ]:
########## Shopping Mall Accessibility Feature Engineering (Shared Distance Matrix + TopK Aggregation) ##########
from tqdm import tqdm

malls = pd.read_csv("auxiliary-data/sg-shopping-malls.csv")
malls["LATITUDE"] = pd.to_numeric(malls["LATITUDE"], errors="coerce")
malls["LONGITUDE"] = pd.to_numeric(malls["LONGITUDE"], errors="coerce")
malls = malls.dropna(subset=["LATITUDE", "LONGITUDE"]).reset_index(drop=True)

lat_col, lon_col = "LATITUDE", "LONGITUDE"

def compute_distance_matrix(df, ref_df, lat_col="LATITUDE", lon_col="LONGITUDE"):
    dist_matrix = np.zeros((len(df), len(ref_df)))
    for i, row in tqdm(enumerate(df.itertuples()), total=len(df), desc=f"Computing {len(df)}×{len(ref_df)} mall distances"):
        lat, lon = getattr(row, lat_col), getattr(row, lon_col)
        dists = np.sqrt((ref_df["LATITUDE"] - lat)**2 + (ref_df["LONGITUDE"] - lon)**2)
        dist_matrix[i, :] = dists.values
    return dist_matrix


print("Computing distance matrices for train and test...")
dist_train_mall = compute_distance_matrix(df_train, malls)
dist_test_mall  = compute_distance_matrix(df_test,  malls)
print("Distance matrices computed once.\n")

def add_basic_mall_features(df, dist_matrix):
    thres_500 = 0.0045
    thres_1000 = 0.009
    df = df.copy()

    df["DIST_NEAREST_MALL"] = dist_matrix.min(axis=1)
    df["DIST_SECOND_MALL"]  = np.partition(dist_matrix, 1, axis=1)[:, 1]
    df["NUM_MALL_500M"]     = (dist_matrix < thres_500).sum(axis=1)
    df["NUM_MALL_1000M"]    = (dist_matrix < thres_1000).sum(axis=1)

    return df


df_train = add_basic_mall_features(df_train, dist_train_mall)
df_test  = add_basic_mall_features(df_test,  dist_test_mall)
print("Basic mall features added.\n")

print("Computing mall–price correlations (train only)...")
price = df_train["RESALE_PRICE"].values
inv_dist_train = 1 / (dist_train_mall + 1e-8)
corrs = [np.corrcoef(inv_dist_train[:, j], price)[0, 1] for j in range(len(malls))]
malls["CORR_WITH_PRICE"] = corrs

malls_sorted = malls.sort_values("CORR_WITH_PRICE", ascending=False).reset_index(drop=True)
print("\nTop 10 malls most positively correlated with resale price:")
print(malls_sorted.head(10)[["NAME", "CORR_WITH_PRICE"]])


def add_nearest_to_topk_mall(df, dist_matrix, malls_sorted):
    df = df.copy()
    top3_idx = malls_sorted.index[:3]
    top5_idx = malls_sorted.index[:5]
    top10_idx = malls_sorted.index[:10]

    df["DIST_MALL_NEAREST_TOP3"]  = dist_matrix[:, top3_idx].min(axis=1)
    df["DIST_MALL_NEAREST_TOP5"]  = dist_matrix[:, top5_idx].min(axis=1)
    df["DIST_MALL_NEAREST_TOP10"] = dist_matrix[:, top10_idx].min(axis=1)
    return df

df_train = add_nearest_to_topk_mall(df_train, dist_train_mall, malls_sorted)
df_test  = add_nearest_to_topk_mall(df_test,  dist_test_mall,  malls_sorted)

print("Added nearest-distance-to-top3/5/10 mall features.\n")

print(df_train[[
    "DIST_NEAREST_MALL", "DIST_SECOND_MALL",
    "NUM_MALL_500M", "NUM_MALL_1000M",
    "DIST_MALL_NEAREST_TOP3", "DIST_MALL_NEAREST_TOP5", "DIST_MALL_NEAREST_TOP10"
]].head())
print(df_test[[
    "DIST_NEAREST_MALL", "DIST_SECOND_MALL",
    "NUM_MALL_500M", "NUM_MALL_1000M",
    "DIST_MALL_NEAREST_TOP3", "DIST_MALL_NEAREST_TOP5", "DIST_MALL_NEAREST_TOP10"
]].head())


### Norm Distance

In [ ]:
dist_cols = [c for c in df_train.columns if c.startswith("DIST_")]

for col in dist_cols:
    if col in df_train.columns:
        min_val = df_train[col].min()
        max_val = df_train[col].max()
        df_train[f"{col}_REV"] = 1 - (df_train[col] - min_val) / (max_val - min_val)
        df_test[f"{col}_REV"]  = 1 - (df_test[col]  - min_val) / (max_val - min_val)

print("Reverse-normalized distance features added:")
print([f"{c}_REV" for c in dist_cols if c in df_train.columns])

Reverse-normalized distance features added:
['DIST_NEAREST_HAWKER_REV', 'DIST_SECOND_HAWKER_REV', 'DIST_NEAREST_MRT_OPEN_REV', 'DIST_SECOND_MRT_OPEN_REV', 'DIST_NEAREST_SCHOOL_REV', 'DIST_SECOND_SCHOOL_REV', 'DIST_NEAREST_SECONDARY_REV', 'DIST_SECOND_SECONDARY_REV', 'DIST_NEAREST_MALL_REV', 'DIST_SECOND_MALL_REV']


In [ ]:
df_train.to_csv("train_processed.csv", index=False)
df_test.to_csv("test_processed.csv", index=False)